In [1]:
import os
from os import listdir
from os.path import isdir, isfile, join
import numpy as np
import cv2 as cv
from keras.utils import to_categorical
from sklearn.model_selection import KFold

Using TensorFlow backend.


In [46]:
path = 'dataset/images/digit'
dir_output = 'dataset/bin/digit 227x227 2.5k'
output = 'dataset'
jumlah_citra = 250
resize = (227, 227)
shuffle = True
grayscale = False
fold_split = 4

daftar_direktori = [direktori for direktori in os.listdir(path) if isdir(join(path, direktori))]
dataset = []

In [47]:
for index_direktori, direktori in enumerate(daftar_direktori):
    daftar_citra = [join(join(path, direktori), citra) for citra in os.listdir(join(path, direktori)) if isfile(join(join(path, direktori), citra))]
    
    if shuffle:
        daftar_citra = np.random.choice(daftar_citra, jumlah_citra, False)
    
    maks_citra = range(len(daftar_citra))
    
    if jumlah_citra > 0:
        maks_citra = range(0, jumlah_citra)
    
    for i in maks_citra:
        citra = cv.imread(daftar_citra[i])
        
        if resize is not False:
            citra = cv.resize(citra, resize)
            
        if grayscale:
            citra = cv.cvtColor(citra, cv.COLOR_BGR2GRAY)
        
        target = index_direktori
        target_categorical = to_categorical(target, len(daftar_direktori))
        
        dataset.append(
            np.array([
                citra, target, target_categorical, daftar_citra[i]
            ])
        )
        
dataset = np.array(dataset)

# KFold

In [48]:
jumlah_kelas = len(daftar_direktori)
jumlah_citra_per_kelas = jumlah_citra
kf = KFold(n_splits = fold_split, shuffle = True)
folds = []

for i in range(fold_split):
    folds.append([[],[]])

# Melakukan k-fold untuk setiap kelas
for i in range(jumlah_kelas - 1):
    start_idx = i * jumlah_citra_per_kelas
    end_idx = (i + 1) * jumlah_citra_per_kelas
    
    fold_iter = 0
    for train, test in kf.split(dataset[start_idx:end_idx]):
        for tr in train:
            folds[fold_iter][0].append(int(tr + start_idx))
        
        for te in test:
            folds[fold_iter][1].append(int(te + start_idx))
        fold_iter += 1
    
folds = np.array(folds)

In [49]:
dataset.shape

(2500, 4)

In [50]:
np.save(f'{join(dir_output, output)}.npy', dataset)
np.save('{0}'.format(join(dir_output, 'folds.npy')), folds)
np.save('{0}'.format(join(dir_output, 'target.npy')), daftar_direktori)

# Load Dataset

In [14]:
dataset = np.load('dataset/bin/ghim20 3k/dataset.npy')

In [15]:
dataset.shape

(3000, 3)

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize = (25, 25))

for i in range(20):
    y = plt.subplot(5, 5, i + 1)
    y.imshow(cv.cvtColor(dataset[i, 0], cv.COLOR_BGR2RGB))
    plt.xticks([])
    plt.yticks([])
    
plt.show()

# Load Folds

In [24]:
folds = np.load('dataset/bin/digit 32x32 50k/folds.npy')

In [25]:
folds.shape

(4, 2)

In [ ]:
folds